# Goal: Fairness Metrics Computation

The goal of this notebook is to computes statistical fairness metrics, after the execution of a set of mitigation techniques, given:
- a training dataset (mitigated in case of a pre-processing technique),
- a target variable,
- a sensible attribute.


# Import Libraries





In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [6]:
import pickle
from tqdm.notebook import tqdm
from source.utils.config import *
from source.utils.metrics_utils import *

# Configure the notebook


Modify the variable in the next code cell to configure the notebook, insert in `dataset_list` the name of the dataset.

In [8]:
dataset_list = ['diabetes-women', 'stroke-prediction', 'diabetes-prediction', 'sepsis', 'aids'] # 'aids', 'myocardial-infarction', 'alzheimer-disease',
#options: "diabetes-women", "sepsis", 'aids', "myocardial-infarction", 'alzheimer-disease', "diabetes-prediction", "stroke-prediction"

# Compute metrics

In [9]:
for dataset_name in dataset_list:
  dataset_config = datasets_config[dataset_name]
  ignore_cols = dataset_config['ignore_cols']
  target_variable = dataset_config['target_variable']
  target_variable_labels = dataset_config['target_variable_labels']
  sensible_attributes = dataset_config['sensible_attributes']
  default_mappings = dataset_config.get('default_mappings', {})
  n_splits = dataset_config['n_splits']
  for sensible_attribute in sensible_attributes:
    for mitigation in all_techniques:
      print(dataset_name, sensible_attribute, mitigation)

      if mitigation == 'original':
        dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
        df = pd.read_csv(dataset_path)
        predictions_and_tests = compute_predictions_and_tests(df, sensible_attribute, target_variable, n_splits, models)
      else:
        # Load the correct source dataset, considering that pre-processing techniques
        # modify the original dataset, while in- and post- processing do not
        if mitigation in new_dataset_mitigations:
          dataset_path = path_to_project + '/data/mitigated/mitigated-{}-{}-{}.csv'.format(dataset_name, sensible_attribute, mitigation) if IN_COLAB else 'data/mitigated/mitigated-{}-{}.csv'.format(dataset_name, mitigation)
        else:
          dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)

        load_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
        with open(load_path, 'rb') as fp:
            predictions_and_tests = pickle.load(fp)

      df = pd.read_csv(dataset_path)
      #df = df.drop(columns=ignore_cols)
      #feature_cols = df.columns
      #sensible_values = [0, 1]  # 0 is the discriminated group, 1 the privileged one
      if mitigation == 'aif360-eo':
        n_splits = aif_config[dataset_name][sensible_attribute]['params']['n_splits_eo']
      else:
         n_splits = dataset_config['n_splits']

      performance_metrics = compute_performance_metrics(predictions_and_tests, models, n_splits, mitigation)
      #Save performance metrics
      save_path = path_to_project + '/measurements/performance_metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
      with open(save_path, 'wb') as fp:
          pickle.dump(performance_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

      final_fairness_metrics = compute_final_fairness_metrics(mitigation, predictions_and_tests, target_variable_labels, models, n_splits)

      # print("DIVISION")
      # for metric in fairness_catalogue:
      #   print(metric)
      #   if mitigation in without_model_mitigations:
      #     print(final_fairness_metrics["division"][metric][0])
      #   else:
      #     for m in models:
      #       print(m, final_fairness_metrics["division"][m][metric][0])
      #   print('---------------')

      # print("SUBTRACTION")
      # for metric in fairness_catalogue:
      #   print(metric)
      #   if mitigation in without_model_mitigations:
      #     print(final_fairness_metrics["subtraction"][metric][0])
      #   else:
      #     for m in models:
      #       print(m, final_fairness_metrics["subtraction"][m][metric][0])
      #   print('---------------')
      #Save the metrics results
      save_path = path_to_project + '/measurements/metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
      with open(save_path, 'wb') as fp:
          pickle.dump(final_fairness_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)



diabetes-women AgeCategory original


  0%|          | 0/6 [00:00<?, ?it/s]

Logistic Regression {'GroupFairness': [0.1834850096287018, 0.017943960188798676], 'PredictiveParity': [0.10414480643928774, 0.03775651729849421], 'PredictiveEquality': [0.15901368870688698, 0.13128915568102523], 'EqualOpportunity': [-0.876688059296755, 0.16159530717239812], 'EqualizedOdds': [0.0813258750592886, 0.04575429623387461], 'ConditionalUseAccuracyEquality': [0.023109445810347323, 0.0766881399199534], 'OverallAccuracyEquality': [0.10536546799811342, 0.03237485088329599], 'TreatmentEquality': [-0.813401187446989, 0.2289611362339803], 'FORParity': [-0.5199999999999999, 0.5900508452667448], 'FN': [-0.8331375560977079, 0.2122758701668953], 'FP': [-0.33909570681528883, 0.23012488121176025]}
Decision Tree {'GroupFairness': [0.17693617748962803, 0.03229438614315287], 'PredictiveParity': [0.11086558570730107, 0.032725468299247426], 'PredictiveEquality': [0.22331402330652989, 0.10281694542499382], 'EqualOpportunity': [-0.39987055285202205, 0.16256811573886806], 'EqualizedOdds': [0.10545

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  0%|          | 0/6 [00:00<?, ?it/s]

...

/content/drive/MyDrive/FairAlgorithm/source/utils/metrics_utils.py:355: RuntimeWarning: invalid value encountered in scalar divide
  elif (TP_priv/(TP_priv+FP_priv)) == 0:
/content/drive/MyDrive/FairAlgorithm/source/utils/metrics_utils.py:358: RuntimeWarning: invalid value encountered in scalar divide
  ConditionalUseAccuracyEquality1 = ((TP_discr/(TP_discr+FP_discr)) / (TP_priv/(TP_priv+FP_priv)))


Logistic Regression {'GroupFairness': [0.33333333333333337, 5.551115123125783e-17], 'PredictiveParity': [0.33333333333333337, 5.551115123125783e-17], 'PredictiveEquality': [0.33333333333333337, 5.551115123125783e-17], 'EqualOpportunity': [-0.4, 0.48989794855663565], 'EqualizedOdds': [0.21904761904761902, 0.09331389496316868], 'ConditionalUseAccuracyEquality': [0.33333333333333337, 5.551115123125783e-17], 'OverallAccuracyEquality': [0.010308847687129717, 0.012376107072676031], 'TreatmentEquality': [0.33333333333333337, 5.551115123125783e-17], 'FORParity': [-0.40231988477786745, 0.5615748736667361], 'FN': [-0.40231988477786745, 0.5615748736667361], 'FP': [0.33333333333333337, 5.551115123125783e-17]}
Decision Tree {'GroupFairness': [-0.5278217905591386, 0.5380512627323161], 'PredictiveParity': [-0.4931746031746032, 0.5109346289021109], 'PredictiveEquality': [-0.0561797752808989, 0.6186377134158404], 'EqualOpportunity': [0.19999999999999996, 0.4], 'EqualizedOdds': [0.2798771121351767, 0.08

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression {'GroupFairness': [0.001199330324092483, 0.032094137043916005], 'PredictiveParity': [-0.005594275510933123, 0.02464733908649569], 'PredictiveEquality': [0.09903869066394819, 0.15139819494606802], 'EqualOpportunity': [0.24689344041244712, 0.2602315273140112], 'EqualizedOdds': [0.041361185933840326, 0.06554015038688474], 'ConditionalUseAccuracyEquality': [-0.05976325196520259, 0.047001050485673365], 'OverallAccuracyEquality': [-0.02011822458144861, 0.03022908208315191], 'TreatmentEquality': [0.19520316927632636, 0.3091691230892075], 'FORParity': [0.2707452806245858, 0.20061383840008742], 'FN': [0.2582830287592041, 0.2542878994072007], 'FP': [0.06079060971093647, 0.19689297125213565]}
Decision Tree {'GroupFairness': [-0.013147719247397821, 0.04391320306640809], 'PredictiveParity': [0.010310067546851912, 0.015448084708519723], 'PredictiveEquality': [-0.062045638263477, 0.13716261191808293], 'EqualOpportunity': [0.14676643281920723, 0.25804872550062774], 'EqualizedOdds':